### Web scraper to scrape indeed.com page job listings

In [3]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [4]:
# function to Extract the Html from indeed
def Extract(page=0):
    scraper = cloudscraper.create_scraper(delay=10, browser="chrome")
    url = f"https://br.indeed.com/jobs?q=Cientista%20de%20Dados&start={page}"
    r = scraper.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup, r


print(Extract()[1])


<Response [200]>


In [5]:
# function to Transform the html we extracted into data
def Transform(soup):
    divs = soup.find_all(
        "div", {"class": "job_seen_beacon"}
    )  # locating tags and classes.(jobs, salaries...)
    for div in divs:
        try:
            company = div.find(
                "span", {"class": "companyName"}
            ).text.strip()  # extracting company name
        except:
            company = ""

        try:
            job_title = div.find(
                "h2", attrs={"class": "jobTitle"}
            ).text.strip()  # extracting job title
        except:
            job_title = ""

        try:
            salary = div.find(
                "div", {"class": ["attribute_snippet", "estimated-salary"]}
            ).text.strip()  # extracting salary
        except:
            salary = ""

        try:
            location = div.find(
                "div", {"class": "companyLocation"}
            ).text.strip()  # extracting location
        except:
            location = ""

        try:
            rating = div.find(
                "span", attrs={"class": "ratingNumber"}
            ).text.strip()  # extracting rating
        except:
            rating = ""

        try:
            summary = div.find(
                "div", attrs={"class": "job-snippet"}
            ).text.strip()  # extracting summary
        except:
            summary = ""
        # grab the results and putting it into a dictionary
        job = {
            "Título": job_title,
            "Empresa": company,
            "Local": location,
            "Salário ": salary,
            "Avaliação": rating,
            "Descrição": summary,
        }
        # appending the data to a list to iterate through it
        joblist.append(job)
    return


joblist = []


In [6]:
for pages in range(0, 35):  # run throw the pages
    info = Extract(pages)[0]
    Transform(info)


In [7]:
# creating a dataframe from the results
df = pd.DataFrame(joblist)
df

,Título,Empresa,Local,Salário,Avaliação,Descrição
0,Cientista de Dados,Criart Serviços de Terceirização de Mão de Obr...,"Belo Horizonte, MG",R$ 9.500 por mês,,"Experiência em estatística descritiva, modelos..."
1,Cientista de Dados,innolevels,Home office in Remoto+1 location,R$ 10.000 - R$ 12.000 por mês,,"Classificação, regressão, clustering, redes ne..."
2,Cientista de Dados,Intera,"Home office in São Paulo, SP",Tempo integral +1,,Experiência com pacotes de análise de dados (P...
3,Analista de Dados / GCP,Innolevels,Home office in Remoto,R$ 5.000 - R$ 10.000 por mês,,Candidate-se a nossa vaga que teremos o grande...
4,Pessoa Cientista de Dados Pleno,Aquarela Analytics,Home office,,,Conhecimento em vários tipos de banco de dados...
...,...,...,...,...,...,...
535,Data Scientist,Avenue Code,Home office in Brasil,,,Estamos procurando um profissional motivado pa...
536,Cientista de Dados ( Machine Learning ) - Vaga...,ArcelorMittal,"Serra, ES",,,Pós-graduação em áreas correlatas com ciências...
537,Analista de dados (Estatístico) Home Office,Scopo Consultoria,"Home office in Natal, RN",R$ 2.000 - R$ 2.500 por mês,,Salário à Combinar + Auxílio Home office + Pla...
538,Cientista de Dados,LEAN SALES,"Itajaí, SC",,,Buscar dados oficiais governamentais e validar...


In [8]:
df.to_excel("data/indeed_jobs.xlsx", sheet_name="Vagas Dados")
